<h1>ACDC Main Demo</h1>

<p>This notebook (which doubles as a script) shows several use cases of ACDC</p>

<p>The codebase is built on top of https://github.com/neelnanda-io/TransformerLens (source version)</p>

<h3>Setup:</h3>
<p>Janky code to do different setup when run in a Colab notebook vs VSCode (adapted from e.g <a href="https://github.com/neelnanda-io/TransformerLens/blob/5c89b7583e73ce96db5e46ef86a14b15f303dde6/demos/Activation_Patching_in_TL_Demo.ipynb">this notebook</a>)</p>

In [58]:
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")

    import subprocess # to install graphviz dependencies
    command = ['apt-get', 'install', 'graphviz-dev']
    subprocess.run(command, check=True)

    import os # make images folder
    os.mkdir("ims/")

    from IPython import get_ipython
    ipython = get_ipython()

    ipython.run_line_magic( # install ACDC
        "pip",
        "install git+https://github.com/ArthurConmy/Automatic-Circuit-Discovery.git@9d5844a",
    )

except Exception as e:
    IN_COLAB = False
    print("Running as a outside of colab")

    import numpy # crucial to not get cursed error
    import plotly

    plotly.io.renderers.default = "colab"  # added by Arthur so running as a .py notebook with #%% generates .ipynb notebooks that display in colab
    # disable this option when developing rather than generating notebook outputs

    import os # make images folder
    if not os.path.exists("ims/"):
        os.mkdir("ims/")

    from IPython import get_ipython

    ipython = get_ipython()
    if ipython is not None:
        print("Running as a notebook")
        ipython.run_line_magic("load_ext", "autoreload")  # type: ignore
        ipython.run_line_magic("autoreload", "2")  # type: ignore
    else:
        print("Running as a script")

Running as a outside of colab
Running as a notebook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h2>Imports etc</h2>

In [59]:
import wandb
import IPython
from IPython.display import Image, display
import torch
import gc
from tqdm import tqdm
import networkx as nx
import os
import torch
import huggingface_hub
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from tqdm import tqdm
import yaml
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from transformer_lens.hook_points import HookedRootModule, HookPoint
from transformer_lens.HookedTransformer import (
    HookedTransformer,
)
try:
    from acdc.tracr_task.utils import (
        get_all_tracr_things,
        get_tracr_model_input_and_tl_model,
    )
except Exception as e:
    print(f"Could not import `tracr` because {e}; the rest of the file should work but you cannot use the tracr tasks")
from acdc.docstring.utils import get_all_docstring_things
from acdc.acdc_utils import (
    make_nd_dict,
    reset_network,
    shuffle_tensor,
    cleanup,
    ct,
    TorchIndex,
    Edge,
    EdgeType,
)  # these introduce several important classes !!!

from acdc.TLACDCCorrespondence import TLACDCCorrespondence
from acdc.TLACDCInterpNode import TLACDCInterpNode
from acdc.TLACDCExperiment import TLACDCExperiment

from acdc.acdc_utils import (
    kl_divergence,
)
from acdc.ioi.utils import (
    get_all_ioi_things,
    get_gpt2_small,
)
from acdc.induction.utils import (
    get_all_induction_things,
    get_validation_data,
    get_good_induction_candidates,
    get_mask_repeat_candidates,
)
from acdc.greaterthan.utils import get_all_greaterthan_things
from acdc.acdc_graphics import (
    build_colorscheme,
    show,
)
import argparse

torch.autograd.set_grad_enabled(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Could not import `tracr` because No module named 'tracr'; the rest of the file should work but you cannot use the tracr tasks


<h2>ACDC Experiment Setup</h2>
<p>We use a `parser to set all the options for the ACDC experiment.
This is still usable in notebooks! We can pass a string to the parser, see below.
We'll reproduce </p>

In [60]:
from transformer_lens import HookedTransformer

# tl_model = HookedTransformer.from_pretrained(
#     "bert-base-cased",
# )

In [61]:

from transformer_lens import HookedTransformer

tl_model = HookedTransformer.from_pretrained(
    # "ai-forever/mGPT",
    # "roneneldan/TinyStories-Instruct-33M",
    "gpt2",
)
tl_model.set_use_attn_result(True)
tl_model.set_use_split_qkv_input(True)
tl_model.to("cuda")

official_model_name gpt2
cfg HookedTransformerConfig:
{'act_fn': 'gelu_new',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 64,
 'd_mlp': 3072,
 'd_model': 768,
 'd_vocab': 50257,
 'd_vocab_out': 50257,
 'device': 'cuda',
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.02886751345948129,
 'model_name': 'gpt2',
 'n_ctx': 1024,
 'n_devices': 1,
 'n_heads': 12,
 'n_layers': 12,
 'n_params': 84934656,
 'normalization_type': 'LNPre',
 'original_architecture': 'GPT2LMHeadModel',
 'parallel_attn_mlp': False,
 'positional_embedding_type': 'standard',
 'rotary_dim': None,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tokenizer_name': 'gpt2',
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_hook_tokens': False,
 'use_local_attn': False,
 'use_split_qk

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      

In [62]:
import torch
import torch.nn.functional as F
race = "chinese" #prostitute
race = "asian" #waitress
race = "russian" #prostitute
race = "indian" #prostitute
race = "polish" #model
race = "dutch" #prostitute
race = "american" #prostitute
race_list = ["chinese", "asian", "russian", "indian", "polish", "dutch", "american", "czech"]
race_list = ["chinese", "asian", "russian", "indian", "polish", "dutch", "american", "czech", "afghan", "albanian", "algerian", "andorran", "angolan", "antiguan", "argentine", "armenian", "australian", "austrian", "azerbaijani", "bahamian", "bahraini", "bangladeshi", "barbadian", "belarusian", "belgian", "belizean", "beninese", "bermudian", "bhutanese", "bolivian", "bosnian", "botswanan", "brazilian", "british", "bruneian", "bulgarian", "burkinan", "burmese", "burundian", "cambodian", "cameroonian", "canadian", "cape verdean", "cayman islander", "central african", "chadian", "chilean","colombian","comoran","congolese","costa rican","croatian","cuban","cypriot","danish","djiboutian","dominican","east timorese","ecuadorean","egyptian","emirati","equatorial guinean","eritrean","estonian","ethiopian","faroese","fijian","filipino","finnish","french","gabonese","gambian","georgian","german","ghanaian","gibraltarian","greek","greenlandic","grenadian","guamanian","guatemalan","guinean","guyanese"]
genders = [("he", "man"), ("she", "woman")]
profession_dict = {}
for race in race_list:
    for pronoun, gender in genders:
        sentences = [f"After spending some time with {race} {gender}, I think {pronoun} might be working as a"]
        sentences = [f"After spending some time with {race} {gender} in Oslo, I think {pronoun} might be working as a"]
        tokens = tl_model.to_tokens(sentences, prepend_bos = True)
        predicted_profession = tl_model.to_str_tokens([torch.argmax(F.log_softmax(tl_model(tokens)[:, -1], dim=-1))])[0][0]
        if(race not in profession_dict):
           profession_dict[race] = {}
        # if(gender not in profession_dict[race]):
        profession_dict[race][gender] = predicted_profession

In [63]:
sentences = [f"John and Kelly tried to go for a swim, "]
# sentences = [f"I just met John. Where is"]
# sentences = [f"Mike nada muy rapido. Si tengo que elegir alguien para mi equipo, le voy a elegir"]
# sentences = [f"one plus three equals four. one plus five equals six. one plus seven equals"]
sentences = [f"Sentence: I am doing good. Sentiment: positive. Sentence: I am not doing bad. Sentiment:"]

tokens = tl_model.to_tokens(sentences, prepend_bos = True)
predicted_country = tl_model.to_str_tokens([torch.argmax(F.log_softmax(tl_model(tokens)[:, -1], dim=-1))])[0][0]
print(predicted_country)

 positive


In [64]:
# Load imdb sentiment analysis dataset
import pandas as pd
tweets = pd.read_csv('Tweets.csv')
sentences = tweets['text'].tolist()
sentiments = tweets['sentiment'].tolist()

# Select only positive and negative tweets
sentences = [sentences[i] for i in range(len(sentences)) if sentiments[i] == 'positive' or sentiments[i] == 'negative']
sentiments = [sentiments[i] for i in range(len(sentiments)) if sentiments[i] == 'positive' or sentiments[i] == 'negative']

In [102]:
# Construct one shot dataset and test the performance of gpt2 model
one_shot_dataset = []
one_shot_sentiments = []
temp_sent = ""
for i in range(len(sentences)):
    if(len(temp_sent)>1):
        temp_sent += " " + sentences[i] + ". Sentiment:"
        one_shot_dataset.append(temp_sent)
        temp_sent = ""
        one_shot_sentiments.append(sentiments[i])
    else:
        temp_sent = f"Sentence: {sentences[i]}. Sentiment: {sentiments[i]}."
    

In [81]:
# Construct two shot dataset and test the performance of gpt2 model
two_shot_dataset = []
two_shot_sentiments = []
temp_sent = ""
num_sentences = 0
for i in range(len(sentences)):
    if(num_sentences>1):
        temp_sent += "Sentence: " + sentences[i] + ". Sentiment:"
        two_shot_dataset.append(temp_sent)
        temp_sent = ""
        two_shot_sentiments.append(sentiments[i])
        num_sentences = 0
    else:
        temp_sent += f"Sentence: {sentences[i]}. Sentiment: {sentiments[i]}. "
        num_sentences+=1

In [97]:
reference_sentence = ""#f"Sentence: {sentences[0]}. Sentiment: {sentiments[0]}. "
one_shot_dataset = []
one_shot_sentiments = []
for i in range(1, len(sentences)):
    one_shot_dataset.append(reference_sentence + f"Sentence: {sentences[i]}. Sentiment:")
    one_shot_sentiments.append(sentiments[i])

In [103]:
# one_shot_dataset[0]
nums = 100
predicted_sentiments = []
for i in one_shot_dataset[:nums]:
# for i in two_shot_dataset[:nums]:
    tokens = tl_model.to_tokens(i, prepend_bos = True)
    predicted_sentiment = tl_model.to_str_tokens([torch.argmax(F.log_softmax(tl_model(tokens)[:, -1], dim=-1))])
    predicted_sentiments.append(predicted_sentiment[0][0][1:])

In [3]:
import numpy as np
one_shot_dataset = np.load('one_shot_sent.npy', allow_pickle=True)
one_shot_dataset

array(['Sentence:  Sooo SAD I will miss you here in San Diego!!!. Sentiment: negative. my boss is bullying me.... Sentiment:',
       'Sentence:  what interview! leave me alone. Sentiment: negative.  Sons of ****, why couldn`t they put them on the releases we already bought. Sentiment:',
       'Sentence: 2am feedings for the baby are fun when he is all smiles and coos. Sentiment: positive.  Journey!? Wow... u just became cooler.  hehe... (is that possible!?). Sentiment:',
       'Sentence: I really really like the song Love Story by Taylor Swift. Sentiment: positive. My Sharpie is running DANGERously low on ink. Sentiment:',
       'Sentence: i want to go to music tonight but i lost my voice.. Sentiment: negative. Uh oh, I am sunburned. Sentiment:',
       'Sentence:  S`ok, trying to plot alternatives as we speak *sigh*. Sentiment: negative. i`ve been sick for the past few days  and thus, my hair looks wierd.  if i didnt have a hat on it would look... http://tinyurl.com/mnf4kw. Sentim

In [107]:
np.save('one_shot_sent.npy',one_shot_dataset[:nums])
np.save('one_shot_sentiments.npy',one_shot_sentiments[:nums])

In [105]:
import numpy as np
np.mean(np.array(predicted_sentiments) == one_shot_sentiments[:nums])

0.62

In [111]:
# ['positive' for i in one_shot_sentiments[:nums] if i == 'positive']
wrongs = []
for i in one_shot_sentiments[:nums]:
    if(i == 'positive'):
        wrongs.append('negative')
    else:
        wrongs.append('positive')
np.save('one_shot_wrong_sentiments.npy',wrongs)

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(profession_dict).T#.to_csv("professions.csv")

,man,woman
chinese,a,a
asian,the,the
russian,a,a
indian,the,the
polish,the,the
...,...,...
grenadian,a,a
guamanian,living,living
guatemalan,the,the
guinean,a,a


In [ ]:
name = "Miguel"
sentences = [f"Así que {name} es un gran amigo, ".format(name)]
tokens = tl_model.to_tokens(sentences, prepend_bos = True)
# tl_model.to_tokens([" he"], prepend_bos = True)
tl_model.to_str_tokens([torch.argmax(F.log_softmax(tl_model(tokens)[:, -1], dim=-1))])

[['iba']]

In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pratikay (uva_dl). Use `wandb login --relogin` to force relogin


acc,▁▇▇▇█▇▇▇
loss,█▃▄▄▁▃▁▁
acc,0.76533
loss,0.16894


In [ ]:
parser = argparse.ArgumentParser(description="Used to launch ACDC runs. Only task and threshold are required")

task_choices = ['ioi', 'docstring', 'induction', 'tracr-reverse', 'tracr-proportion', 'greaterthan']
parser.add_argument('--task', type=str, required=True, choices=task_choices, help=f'Choose a task from the available options: {task_choices}')
parser.add_argument('--threshold', type=float, required=True, help='Value for THRESHOLD')
parser.add_argument('--first-cache-cpu', type=str, required=False, default="True", help='Value for FIRST_CACHE_CPU (the old name for the `online_cache`)')
parser.add_argument('--second-cache-cpu', type=str, required=False, default="True", help='Value for SECOND_CACHE_CPU (the old name for the `corrupted_cache`)')
parser.add_argument('--zero-ablation', action='store_true', help='Use zero ablation')
parser.add_argument('--using-wandb', action='store_true', help='Use wandb')
parser.add_argument('--wandb-entity-name', type=str, required=False, default="remix_school-of-rock", help='Value for WANDB_ENTITY_NAME')
parser.add_argument('--wandb-group-name', type=str, required=False, default="mi-proj", help='Value for WANDB_GROUP_NAME')
parser.add_argument('--wandb-project-name', type=str, required=False, default="mi-proj", help='Value for WANDB_PROJECT_NAME')
parser.add_argument('--wandb-run-name', type=str, required=False, default=None, help='Value for WANDB_RUN_NAME')
parser.add_argument("--wandb-dir", type=str, default="/tmp/wandb")
parser.add_argument("--wandb-mode", type=str, default="online")
parser.add_argument('--indices-mode', type=str, default="normal")
parser.add_argument('--names-mode', type=str, default="normal")
parser.add_argument('--device', type=str, default="cuda")
parser.add_argument('--reset-network', type=int, default=0, help="Whether to reset the network we're operating on before running interp on it")
parser.add_argument('--metric', type=str, default="kl_div", help="Which metric to use for the experiment")
parser.add_argument('--torch-num-threads', type=int, default=0, help="How many threads to use for torch (0=all)")
parser.add_argument('--seed', type=int, default=1234)
parser.add_argument("--max-num-epochs",type=int, default=100_000)
parser.add_argument('--single-step', action='store_true', help='Use single step, mostly for testing')

if ipython is not None:
    # we are in a notebook
    # you can put the command you would like to run as the ... in r"""..."""
    args = parser.parse_args(
        [line.strip() for line in r"""--task=docstring\
--zero-ablation\
--threshold=0.71\
--indices-mode=reverse\
--first-cache-cpu=False\
--second-cache-cpu=False\
--max-num-epochs=100000\
--wandb-run-name=eng1\
--using-wandb""".split("\\\n")]
    )
else:
    # read from command line
    args = parser.parse_args()

# process args

if args.torch_num_threads > 0:
    torch.set_num_threads(args.torch_num_threads)
torch.manual_seed(args.seed)

TASK = args.task
if args.first_cache_cpu is None: # manage default
    ONLINE_CACHE_CPU = True
elif args.first_cache_cpu.lower() == "false":
    ONLINE_CACHE_CPU = False
elif args.first_cache_cpu.lower() == "true":
    ONLINE_CACHE_CPU = True
else: 
    raise ValueError(f"first_cache_cpu must be either True or False, got {args.first_cache_cpu}")
if args.second_cache_cpu is None:
    CORRUPTED_CACHE_CPU = True
elif args.second_cache_cpu.lower() == "false":
    CORRUPTED_CACHE_CPU = False
elif args.second_cache_cpu.lower() == "true":
    CORRUPTED_CACHE_CPU = True
else:
    raise ValueError(f"second_cache_cpu must be either True or False, got {args.second_cache_cpu}")
THRESHOLD = args.threshold  # only used if >= 0.0
ZERO_ABLATION = True if args.zero_ablation else False
USING_WANDB = True if args.using_wandb else False
WANDB_ENTITY_NAME = args.wandb_entity_name
WANDB_PROJECT_NAME = args.wandb_project_name
WANDB_RUN_NAME = args.wandb_run_name
WANDB_GROUP_NAME = args.wandb_group_name
INDICES_MODE = args.indices_mode
NAMES_MODE = args.names_mode
DEVICE = args.device
RESET_NETWORK = args.reset_network
SINGLE_STEP = True if args.single_step else False

NameError: name 'argparse' is not defined

<h2>Setup Task</h2>

In [ ]:
second_metric = None  # some tasks only have one metric
use_pos_embed = TASK.startswith("tracr")

if TASK == "ioi":
    num_examples = 100
    things = get_all_ioi_things(
        num_examples=num_examples, device=DEVICE, metric_name=args.metric
    )
elif TASK == "tracr-reverse":
    num_examples = 6
    things = get_all_tracr_things(
        task="reverse",
        metric_name=args.metric,
        num_examples=num_examples,
        device=DEVICE,
    )
elif TASK == "tracr-proportion":
    num_examples = 50
    things = get_all_tracr_things(
        task="proportion",
        metric_name=args.metric,
        num_examples=num_examples,
        device=DEVICE,
    )
elif TASK == "induction":
    num_examples = 10 if IN_COLAB else 50
    seq_len = 300
    # TODO initialize the `tl_model` with the right model
    things = get_all_induction_things(
        num_examples=num_examples, seq_len=seq_len, device=DEVICE, metric=args.metric
    )
elif TASK == "docstring":
    num_examples = 50
    seq_len = 41
    things = get_all_docstring_things(
        num_examples=num_examples,
        seq_len=seq_len,
        device=DEVICE,
        metric_name=args.metric,
        correct_incorrect_wandb=True,
    )
elif TASK == "greaterthan":
    num_examples = 100
    things = get_all_greaterthan_things(
        num_examples=num_examples, metric_name=args.metric, device=DEVICE
    )
else:
    raise ValueError(f"Unknown task {TASK}")

Using pad_token, but it is not set yet.


Loaded pretrained model ai-forever/mGPT into HookedTransformer
Moving model to device:  cuda


<p> Let's define the four most important objects for ACDC experiments:

In [ ]:
validation_metric = things.validation_metric # metric we use (e.g KL divergence)
toks_int_values = things.validation_data # clean data x_i
toks_int_values_other = things.validation_patch_data # corrupted data x_i'
tl_model = things.tl_model # transformerlens model

if RESET_NETWORK:
    reset_network(TASK, DEVICE, tl_model)

<h2>Setup ACDC Experiment</h2>

In [ ]:
# Make notes for potential wandb run
try:
    with open(__file__, "r") as f:
        notes = f.read()
except:
    notes = "No notes generated, expected when running in an .ipynb file"

tl_model.reset_hooks()

# Save some mem
gc.collect()
torch.cuda.empty_cache()

# Setup wandb if needed
if WANDB_RUN_NAME is None or IPython.get_ipython() is not None:
    WANDB_RUN_NAME = f"{ct()}{'_randomindices' if INDICES_MODE=='random' else ''}_{THRESHOLD}{'_zero' if ZERO_ABLATION else ''}"
else:
    assert WANDB_RUN_NAME is not None, "I want named runs, always"

tl_model.reset_hooks()
exp = TLACDCExperiment(
    model=tl_model,
    threshold=THRESHOLD,
    using_wandb=USING_WANDB,
    wandb_entity_name=WANDB_ENTITY_NAME,
    wandb_project_name=WANDB_PROJECT_NAME,
    wandb_run_name=WANDB_RUN_NAME,
    wandb_group_name=WANDB_GROUP_NAME,
    wandb_notes=notes,
    wandb_dir=args.wandb_dir,
    wandb_mode=args.wandb_mode,
    wandb_config=args,
    zero_ablation=ZERO_ABLATION,
    ds=toks_int_values,
    ref_ds=toks_int_values_other,
    metric=validation_metric,
    second_metric=second_metric,
    verbose=True,
    indices_mode=INDICES_MODE,
    names_mode=NAMES_MODE,
    corrupted_cache_cpu=CORRUPTED_CACHE_CPU,
    hook_verbose=False,
    online_cache_cpu=ONLINE_CACHE_CPU,
    add_sender_hooks=True,
    use_pos_embed=use_pos_embed,
    add_receiver_hooks=False,
    remove_redundant=False,
    show_full_index=use_pos_embed,
)

dict_keys(['blocks.23.hook_resid_post', 'blocks.23.hook_mlp_out', 'blocks.23.hook_mlp_in', 'blocks.23.attn.hook_result', 'blocks.23.attn.hook_q', 'blocks.23.hook_q_input', 'blocks.23.attn.hook_k', 'blocks.23.hook_k_input', 'blocks.23.attn.hook_v', 'blocks.23.hook_v_input', 'blocks.22.hook_mlp_out', 'blocks.22.hook_mlp_in', 'blocks.22.attn.hook_result', 'blocks.22.attn.hook_q', 'blocks.22.hook_q_input', 'blocks.22.attn.hook_k', 'blocks.22.hook_k_input', 'blocks.22.attn.hook_v', 'blocks.22.hook_v_input', 'blocks.21.hook_mlp_out', 'blocks.21.hook_mlp_in', 'blocks.21.attn.hook_result', 'blocks.21.attn.hook_q', 'blocks.21.hook_q_input', 'blocks.21.attn.hook_k', 'blocks.21.hook_k_input', 'blocks.21.attn.hook_v', 'blocks.21.hook_v_input', 'blocks.20.hook_mlp_out', 'blocks.20.hook_mlp_in', 'blocks.20.attn.hook_result', 'blocks.20.attn.hook_q', 'blocks.20.hook_q_input', 'blocks.20.attn.hook_k', 'blocks.20.hook_k_input', 'blocks.20.attn.hook_v', 'blocks.20.hook_v_input', 'blocks.19.hook_mlp_out'

<h2>Run steps of ACDC: iterate over a NODE in the model's computational graph</h2>
<p>WARNING! This will take a few minutes to run, but there should be rolling nice pictures too : )</p>

In [ ]:
for i in range(args.max_num_epochs):
    exp.step(testing=False)

    show(
        exp.corr,
        f"ims/img_new_{i+1}.png",
        show_full_index=use_pos_embed,
    )

    if IN_COLAB or ipython is not None:
        # so long as we're not running this as a script, show the image!
        display(Image(f"ims/img_new_{i+1}.png"))

    print(i, "-" * 50)
    print(exp.count_no_edges())

    if i == 0:
        exp.save_edges("edges.pkl")

    if exp.current_node is None or SINGLE_STEP:
        break

exp.save_edges("another_final_edges.pkl")

if USING_WANDB:
    edges_fname = f"edges.pth"
    exp.save_edges(edges_fname)
    artifact = wandb.Artifact(edges_fname, type="dataset")
    artifact.add_file(edges_fname)
    wandb.log_artifact(artifact)
    os.remove(edges_fname)
    wandb.finish()

No edge 233029
New metric: 0.0

Node: cur_parent=TLACDCInterpNode(blocks.23.hook_mlp_out, [:]) (self.current_node=TLACDCInterpNode(blocks.23.hook_resid_post, [:]))

Metric after removing connection to blocks.23.hook_mlp_out [:] is 0.055740222334861755 (and current metric 0.0)
Result is 0.055740222334861755...so removing connection
No edge 233028

Node: cur_parent=TLACDCInterpNode(blocks.23.attn.hook_result, [:, :, 0]) (self.current_node=TLACDCInterpNode(blocks.23.hook_resid_post, [:]))

Metric after removing connection to blocks.23.attn.hook_result [:, :, 0] is 0.07561872154474258 (and current metric 0.055740222334861755)
Result is 0.01987849920988083...so removing connection

Node: cur_parent=TLACDCInterpNode(blocks.23.attn.hook_result, [:, :, 1]) (self.current_node=TLACDCInterpNode(blocks.23.hook_resid_post, [:]))

Metric after removing connection to blocks.23.attn.hook_result [:, :, 1] is 0.07102078944444656 (and current metric 0.07561872154474258)
Result is -0.0045979321002960205..

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 for i in range(args.max_num_epochs):                                                        │
│ ❱  2 │   exp.step(testing=False)                                                                 │
│    3 │                                                                                           │
│    4 │   show(                                                                                   │
│    5 │   │   exp.corr,                                                                           │
│                                                                                                  │
│ /home/pratik/.temp/new/Automatic-Circuit-Discovery/acdc/TLACDCExperiment.py:579 in step          │
│                                                                                                  │
│   576 │   │   │   │   if self.second_metric is not None:                                         │
│   577 │   │   │   │   │   old_second_metric = self.cur_second_metric                             │
│   578 │   │   │   │                                                                              │
│ ❱ 579 │   │   │   │   self.update_cur_metric(recalc_edges=False) # warning: gives fast evaluat   │
│   580 │   │   │   │   evaluated_metric = self.cur_metric # self.metric(self.model(self.ds)) #    │
│   581 │   │   │   │                                                                              │
│   582 │   │   │   │   if early_stop: # for debugging the effects of one and only one forward p   │
│                                                                                                  │
│ /home/pratik/.temp/new/Automatic-Circuit-Discovery/acdc/TLACDCExperiment.py:189 in               │
│ update_cur_metric                                                                                │
│                                                                                                  │
│   186 │                                                                                          │
│   187 │   def update_cur_metric(self, recalc_metric=True, recalc_edges=True, initial=False):     │
│   188 │   │   if recalc_metric:                                                                  │
│ ❱ 189 │   │   │   logits = self.model(self.ds)                                                   │
│   190 │   │   │   self.cur_metric = self.metric(logits)                                          │
│   191 │   │   │   if self.second_metric is not None:                                             │
│   192 │   │   │   │   self.cur_second_metric = self.second_metric(logits)                        │
│                                                                                                  │
│ /home/pratik/miniconda3/envs/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1501   │
│ in _call_impl                                                                                    │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                     

<h2>Save the final subgraph of the model</h2>
<p>There are more than `exp.count_no_edges()` here because we include some "placeholder" edges needed to make ACDC work that don't actually matter</p>
<p>Also note that the final image has more than 12 edges, because the edges from a0.0_q and a0.0_k are not connected to the input</p>
<p>We recover minimal induction machinery! `embed -> a0.0_v -> a1.6k`</p>

In [ ]:
exp.save_subgraph(
    return_it=True,
) 

OrderedDict([(('blocks.3.hook_resid_post',
               (None,),
               'blocks.3.attn.hook_result',
               (None, None, 6)),
              True),
             (('blocks.3.hook_resid_post',
               (None,),
               'blocks.3.attn.hook_result',
               (None, None, 0)),
              True),
             (('blocks.3.attn.hook_result',
               (None, None, 6),
               'blocks.3.attn.hook_q',
               (None, None, 6)),
              True),
             (('blocks.3.attn.hook_result',
               (None, None, 6),
               'blocks.3.attn.hook_k',
               (None, None, 6)),
              True),
             (('blocks.3.attn.hook_result',
               (None, None, 6),
               'blocks.3.attn.hook_v',
               (None, None, 6)),
              True),
             (('blocks.3.attn.hook_result',
               (None, None, 0),
               'blocks.3.attn.hook_q',
               (None, None, 0)),
              T